Miscellaneous
=============

**Outline**:
1. The Python standard library
2. Formatting 
3. Tools
4. Questions
5. Closing words

## 1. The Python standard library

Python comes with many interesting modules. Here is a selection:
- `os`/`sys`: system-specific parameters and functions;
- `datetime`: manipulation of date and time (limited support for timezones);
- `logging`: built-in logging facility;
- `pathlib`: path, file OOP manipulation interface;
- `collections`: base classes for collections and a few useful concrete ones;
- `itertools`: tools related to iteration;
- `functools`: functional programming tools (decorators, partial, reduce, etc.);
- `argparse`: built-in CLI argument parsings;
- `math`/`random`: you have guessed it;
- `re`: regular expression.


### Defaultdict

In a `defaultdict`, all values are of the same type. If you search for a key that does not exists, a new default becomes attached to that key.

In [2]:
from collections import defaultdict

data = "ABCABABCB"

histogram = defaultdict(int)

for x in data:
    histogram[x] += 1

histogram

defaultdict(int, {'A': 3, 'B': 4, 'C': 2})

In [4]:
from collections import defaultdict

# load balacing
ppl = [
    "Alice", "Bob", "Claire", "David", "Emma", 
    "Felix", "Grace", "Henry", "Isabelle", "Jack"
]

assignment = defaultdict(list) 
for p in ppl:
    assignment[hash(p) % 3].append(p)

assignment

defaultdict(list,
            {2: ['Alice', 'Emma', 'Grace', 'Isabelle'],
             0: ['Bob', 'Claire', 'Henry', 'Jack'],
             1: ['David', 'Felix']})

### Dispatching (overloading)

In a language like Java, you can have several methods with the same name and different inputs. The proper method is invoked based on the types of the arguments. In Python (which is weakly typed) you do not have this functionality out-of-the-box. The `functools` module provides a dispatching tool that might achieve the same result in some circumstances.

In [6]:
from functools import singledispatchmethod

class Negator:
    @singledispatchmethod
    def neg(self, arg):
        raise NotImplementedError("Cannot negate a")

    @neg.register
    def _(self, arg: int):
        return -arg

    @neg.register
    def _(self, arg: bool):
        return not arg

print(Negator().neg(2))
print(Negator().neg(True))

-2
False


### Itertools

Itertools provides many utilities to create iterables like cycling over a sequence, batching data, creating the cartesian product of iterables, creating an iterable of permutations or combinations, etc.

In [7]:
from itertools import permutations

list(permutations("ABC"))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

## 2. Formatting 

In Python 3.9, formatting string is done via the f-strings. Within the substitution expression of the f-string, you can give formatting options:

In [15]:
s = "blabla"
print(f"'{s:10}'")  # Make sure the length is at least 10
print(f"'{s:>10}'") # same + right aligned
print(f"'{s:^10}'") # same but centered
print(f"{0.123456789:.1f}")  # print as float (f) with 1 decimal (.1)
print(f"{0.123456789:e}")  # print in scientific notation
print(f"{'this is the repr'!r}")
print(f"{'this is the str'}")

'blabla    '
'    blabla'
'  blabla  '
0.1
1.234568e-01
'this is the repr'
this is the str


See https://docs.python.org/3/library/string.html for more.

:skull: It is possible to define formatting options for a custom class by overwriting the `__format__` method.

## 3. Tools

Although the focus of the training was on the design and code aspects, ensuring a successful and lasting project is also about good tooling:
- code versioning (git + repository);
- testing (eg. pytest);
- linting (eg. black);
- type-checking (eg. mypy);
- CI/CD;
- packet management (eg. poetry).

## 4. Questions

:question:


## 5. Closing words 

This training was about the standard library and a few miscellaneous other things.

Note that if there are design patterns, there also exists *anti-pattern*: recipes to common problems that are counterproductive in some way (ineffective, difficult to maintain, etc.)

The trainings were about design patterns in Python, but we did not cover all the design patterns. You can easily research the remaining. As for Python, there is still plenty to explore, such as:
* data access (descriptors, MRO, slots);
* parallel computing (thread/processor pools, async, locks, GIL, etc.);
* multiple inheritance (diamond problem, mixins, MRO, composition over inheritance);
* serialization;
* and more.
